In [1]:
import csv
import json
import sys
import pandas as pd
import os
import copy
import math
import json
from bitmap import BitMap
from collections import defaultdict
import numpy as np
import itertools as it


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz    
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip3 install -q findspark
!pip3 install -q pyspark
!pip3 install -q kaggle
!pip3 install bitmap
from bitmap import BitMap

from google.colab import files
print("Upload kaggle.json to use kaggle APIs")
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

# !curl -OL https://raw.githubusercontent.com/Bertonc98/ProgettoAMD/main/sample.csv

!kaggle datasets files "ashirwadsangwan/imdb-dataset"
!kaggle datasets download "ashirwadsangwan/imdb-dataset" -f "title.principals.tsv.gz"
!unzip "title.principals.tsv.gz.zip"
!rm "title.principals.tsv.gz.zip"
!gunzip -k "title.principals.tsv.gz"
!rm "title.principals.tsv.gz"

     |████████████████████████████████| 212.3MB 61kB/s 
     |████████████████████████████████| 204kB 15.1MB/s 
Upload kaggle.json to use kaggle APIs


Saving kaggle.json to kaggle.json
kaggle.json
name                      size  creationDate         
-----------------------  -----  -------------------  
title.akas.tsv.gz        158MB  2019-11-18 18:36:47  
title.principals.tsv.gz  290MB  2019-11-18 18:36:47  
name.basics.tsv.gz       180MB  2019-11-18 18:36:47  
title.ratings.tsv.gz       5MB  2019-11-18 18:36:47  
title.basics.tsv.gz      108MB  2019-11-18 18:36:47  
 95% 272M/287M [00:01<00:00, 194MB/s]
100% 287M/287M [00:01<00:00, 173MB/s]
Archive:  title.principals.tsv.gz.zip
  inflating: title.principals.tsv.gz  


In [2]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

home = os.environ["SPARK_HOME"]
os.environ["PYTHONPATH"] = home+"/python"+home+"/python/lib/py4j-0.10.9-src.zip:"+os.environ["PYTHONPATH"]

In [3]:
import findspark
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkConf
findspark.init()
conf = SparkConf()
conf.set("spark.driver.memory", "5g")
conf.set("spark.executor.memory", "5g")
spark = SparkSession.builder.config(conf = conf).master("local[*]").getOrCreate()

In [4]:
sourceFile = 'title.principals.tsv'

data = spark.read.options(delimiter = "\t", header=True).csv(sourceFile)

from pyspark.sql.window import Window

from pyspark.sql.functions import monotonically_increasing_id,row_number

# rdd_result = data.select(['nconst', 'category'])\
#             .where("category == 'actress' or category == 'actor' ")\
#             .select(['nconst'])\
#             .distinct()

# rdd_result = rdd_result.withColumn("idx",row_number().over(Window.orderBy(monotonically_increasing_id())))

#ALTERNATIVA
filter_actors = lambda row: row["category"] in ["actor", "actress"]
map_actors = lambda row: (row["tconst"], int(row["nconst"][2:])) 

rdd_basket = data.rdd.filter(filter_actors)\
                 .map(map_actors)\
                 .groupByKey()\
                 .mapValues(list)\
                 .values()

In [ ]:
rdd_basket.take(10)

[[525908],
 [85865],
 [194945, 617588],
 [168621],
 [168621],
 [2940072],
 [617588],
 [168621],
 [177886, 298300, 65035, 250480, 422465, 588295, 1096358, 259860],
 [1908, 1173635, 792727]]

In [ ]:
# sourceFile = 'title.principals.tsv'

# data = spark.read.options(delimiter = "\t", header=True).csv(sourceFile)

# from pyspark.sql.window import Window

# from pyspark.sql.functions import monotonically_increasing_id,row_number




In [ ]:
# this dictionary maps all the items to an int
# itemToInt = {key:value for (key, value) in rdd_result.collect()}

In [ ]:
# filter_actors = lambda row: row["category"] in ["actor", "actress"]

# map_actors = lambda row: (row["tconst"], itemToInt[row["nconst"]]) 

# rdd_basket = data.rdd.filter(filter_actors)\
#                  .map(map_actors)\
#                  .groupByKey()\
#                  .mapValues(list)\
#                  .values()
# res = rdd_basketId.collect()
# with open('preprocessed.csv', 'w', newline='') as f:
#     writer = csv.writer(f, delimiter=";")
#     writer.writerows(res)

In [ ]:
# with open('intToItem.csv', 'w', encoding = "utf-8") as f:
#     writer = csv.writer(f, delimiter=";")
#     writer.writerows(zip(itemToInt.values(), itemToInt.keys()))
# del itemToInt

In [5]:
sample = rdd_basket.sample(False, 0.001)
sample.cache()

PythonRDD[19] at RDD at PythonRDD.scala:53

In [ ]:
sample.count()

3669

In [6]:
#Apriori PCY
def hashF(element, size):
    return hash(element)%size

def get_combinations(basket, size):
    return it.combinations(sorted(basket), size)

def update_frequency(l, item):
    if item + 1 > len(l):
        l.extend((item+1-len(l))*[0])
    l[item] += 1
    
def singleton_pairs_frequency(chunk, hash_size, threshold):
    singleton_freq = []
    hashmap = np.zeros(hash_size)
    for basket in chunk:
        basket = list(map(int, basket))
        for item in basket:
            update_frequency(singleton_freq, item)
        for pair in get_combinations(basket, 2):
            hashmap[hashF(pair, hash_size)] += 1
    return filter_singletons(singleton_freq, threshold), hashmap

# this function filter the given singletons with the absolute freqeuncy, deleting the ones with
# frequency below the threshold
def filter_singletons(singleton, threshold):
    return [(iid,) for iid, freq in enumerate(singleton) if freq>threshold]    

# this function collapse the hashmap given in input into a bitmap of size "size" and set a 1 in the bitmap if the value in
# the hashmap is over the the given threshold
def hash_to_bitmap(hashmap, size, threshold):
    bmap = BitMap(size)
    for position in range(len(hashmap)):
        if hashmap[position] >= threshold:
            bmap.set(position)
    del hashmap
    return bmap

def get_joins(tuples, k):
    #implement join in this way
    #to obtain an element of lenght k:
    #join 2 elements of length k-1 with the first k-1 elements in common
    l = []
    for i in range(len(tuples)-1):
        for j in range(i+1, len(tuples)):
            if(tuples[i][:k-2] == tuples[j][:k-2]):
                l += [tuples[i] + tuples[j][k-2:]]
    return l

def get_kth(chunk, precedents, hash_size, bmap, threshold, k):
#     tuple of length k with corresponding bitmap value = 1
    kth = [tpl for tpl in get_joins(precedents, k) if bmap[hashF(tpl, hash_size)]]
    
#     counts the absolute frequency of the tuple in kth
    counters = np.zeros(len(kth))
#     hashmap for the tuples of length k+1
    next_hashmap = np.zeros(hash_size)
    
    for basket in chunk:
        basket = list(map(int, basket))
        tpls = list(get_combinations(basket, k))
        for i in range(len(kth)):
            if kth[i] in tpls:
                counters[i] += 1
        for kth_tuples in get_combinations(basket, k+1):
            next_hashmap[hashF(kth_tuples, hash_size)] += 1

    return [kth[i] for i in range(len(kth)) if counters[i]>=threshold], next_hashmap


In [7]:
def pcy(threshold, hash_size):
# def main(chunk, threshold, hash_size):
  def main(partition):
      generator, chunk = it.tee(partition, 2)
      frequent_sets = []

  #     phase 1: compute frequency of singletons and pairs; filter nonfrequent singletons
      singleton, hashmap = singleton_pairs_frequency(chunk, hash_size, threshold)
  #     phase 2: collpse the hahsmap into a bitmap
      bitmap = hash_to_bitmap(hashmap, hash_size, threshold)
  #     phase 3
  #     frequent_sets contains all the sets of all the kth length that are frequent
  #     bitmap is the bitmap mapping the sets of length k+1
  #     frequent_sets, bitmap = frequent_couple(data, singleton, hash_size, bitmap, threshold)
      frequent_sets += singleton
      
      k = 2
      result = ["placeholder"]
      while result != []:
          precedents = [i for i in frequent_sets if len(i) == k-1 ]
          chunk = copy.copy(generator)
          result, hashmap = get_kth(chunk, precedents, hash_size, bitmap, threshold, k)
          bitmap = hash_to_bitmap(hashmap, hash_size, threshold)
          frequent_sets = frequent_sets + result
          # print("Found frequent itemset of length: ",k)
          k += 1
          
      return frequent_sets
  return main

In [8]:
# TOIVONEN
# Take a sample of the total, a fraction p€[0,1] (adjust the threshold s = p * s)
# Run Apriori over it and obtain the candidates frequent itemset from the sample
# Build the negative border from candidates
# Scan all the basket and check for itemset in candidates or in negative border:
#   - If no element from the negative border is frequent return candidates as frequent itemsets
#   - Otherwise no answer, need to rerun with different sample or threshold
def get_superset(el, candidates):
  return []

def negative_border(candidates):
  nb = [get_superset(el, candidates) for el in candidates]
  return nb

# sample.filter(lambda basket: basket not in candidates)

p = 1/sample.getNumPartitions() #to fix
threshold = math.ceil(sample.count()/1000)
s_threshold = math.ceil(0.95 * threshold * p)
hash_size = 504 #this because we run PCY to find candidates
candidates = sample.mapPartitions(pcy(s_threshold, hash_size))\
                   .map(lambda item: (1,[item]))\
                   .reduceByKey(lambda a, b: list(set(a + b )))\
                   .values()\
                   .collect()[0]
# nb = negative_border(candidates)

In [10]:
s_threshold

1

In [16]:
nb = sample.flatMap(lambda xs: [x for x in xs])\
           .distinct()\
           .map(lambda el: (el,))\
           .filter(lambda el: el not in candidates)\
           .collect()
print(sample.flatMap(lambda xs: [x for x in xs])\
                .distinct()\
                .count())
len(nb)

12984


12813

In [11]:
# frequent_s = sorted([s[0] for s in candidates if s not in nb and len(s)==1])
# nb += [el for el in list(it.combinations(frequent_s, 2)) if el not in candidates]

diff = sample.flatMap(lambda xs: [x for x in xs])\
             .distinct()\
             .map(lambda el: (el,))\
             .filter(lambda el: el not in candidates)\
             .collect()
nb = diff
prec = sample.flatMap(lambda xs: [(x,) for x in xs])\
                .distinct()\
                .collect()
k=2
while k<3:
  print(k)
  candidates_k = [el for el in candidates if len(el) == k]
  diff = [kth for kth in get_joins(prec, k) if kth not in candidates_k]
  nb += diff
  prec = candidates_k
  k += 1
nb

2


[(178750,),
 (590044,),
 (540774,),
 (421720,),
 (298831,),
 (612144,),
 (406393,),
 (715637,),
 (142571,),
 (553150,),
 (86697,),
 (2206360,),
 (2962570,),
 (82537,),
 (495716,),
 (307411,),
 (1913665,),
 (2212509,),
 (453440,),
 (168454,),
 (4828538,),
 (1210573,),
 (513123,),
 (10924199,),
 (902486,),
 (4494035,),
 (8779797,),
 (5401097,),
 (10364081,),
 (10556923,),
 (11057865,),
 (2753361,),
 (896649,),
 (407069,),
 (278304,),
 (94887,),
 (442,),
 (465127,),
 (1481610,),
 (521495,),
 (32890,),
 (4407741,),
 (707187,),
 (655759,),
 (1527227,),
 (1794663,),
 (2504034,),
 (2734329,),
 (673205,),
 (893581,),
 (240019,),
 (4725513,),
 (5425940,),
 (4680078,),
 (6015724,),
 (6106217,),
 (1756274,),
 (77623,),
 (1483703,),
 (1952613,),
 (4420364,),
 (1036373,),
 (29406,),
 (1067170,),
 (158626,),
 (7168473,),
 (7233200,),
 (4823000,),
 (4187131,),
 (5449483,),
 (317980,),
 (4569474,),
 (1053247,),
 (181766,),
 (592176,),
 (498472,),
 (1954290,),
 (7008651,),
 (243763,),
 (99879,),
 (5664

In [13]:
len(nb)

91158555

In [30]:
len(frequent_s)

171

In [ ]:
# SON

# phase 1 scan all baskets 
# -input: id_movie, list(int_actor)
# -compute frequency (count)
# -count combinations (size 2) into hashmap
# -filter frequent singletons (count >= s)
# -output: int_actor: count

p = 1/sample.getNumPartitions()
#to fix
threshold = math.ceil(sample.count()/1000)
s_threshold = math.ceil(0.95 * threshold * p)
# s_threshold = 1
hash_size = 504
candidates = sample.mapPartitions(pcy(s_threshold, hash_size))\
                   .map(lambda item: (1,[item]))\
                   .reduceByKey(lambda a, b: list(set(a + b )))\
                   .values()\
                   .collect()[0]


In [ ]:
def son(candidates):
  def son_main(partition):
    count = {item: 0 for item in candidates}
    for basket in partition:
      for c in candidates:
          if c in get_combinations(basket, len(c)):
            count[c] += 1
    return [(k, v) for k, v in count.items()]
  return son_main

frequent = sample.mapPartitions(son(candidates))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda item: item[1] >= s)\
                 .map(lambda item: item[0])\

In [ ]:
# f = csv.reader(open("intToItem.csv", encoding="utf-8", newline="\n"), delimiter=";")
# intToItem = { int(k):v for (k, v) in f}

In [ ]:
# def getItemIdFromInt(l):
#   return [intToItem[k] for k in l[0]]
def getItemIdFromInt(l):
  return ["nm" + str(k).zfill(7) if len(str(k))<7 else  "nm" + str(k) for k in l]

frequentId = frequent.map(getItemIdFromInt)

In [ ]:
!kaggle datasets download "ashirwadsangwan/imdb-dataset" -f "name.basics.tsv.gz"
!unzip "name.basics.tsv.gz.zip"
!rm "name.basics.tsv.gz.zip"
!gunzip -k "name.basics.tsv.gz"
!rm "name.basics.tsv.gz"

 94% 170M/180M [00:02<00:00, 102MB/s] 
100% 180M/180M [00:02<00:00, 81.2MB/s]
Archive:  name.basics.tsv.gz.zip
  inflating: name.basics.tsv.gz      


In [ ]:
nameTable = spark.read.options(delimiter = "\t", header=True).csv("name.basics.tsv")
# idName = {id:name for (id, name) in nameTable.select(["nconst","primaryName"]).collect()}

In [ ]:
flattened_frequentId = frequentId.flatMap(lambda xs: [x for x in xs]).collect()

nameTable = nameTable.filter(nameTable.nconst.isin(flattened_frequentId))\
                     .select(["nconst", "primaryName"])\
                     .collect()
nameTable = {id : name for (id, name) in nameTable}

In [ ]:
frequentId = frequentId.collect()
basket_translated = []
for basket in frequentId:
  basket_translated.append([nameTable[id] for id in basket])
basket_translated